In [44]:
import ujson
import string
from tqdm.notebook import tqdm

from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

from table_bert import Table
from table_bert.dataset import Example

In [52]:
commoncrawl_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/TaBERT_datasets/CommonCrawl/common_crawl.preprocessed.jsonl'
wikitables_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/TaBERT_datasets/WikiTables/wiki_tables.jsonl'

commoncrawl_n_lines = 4986739
wikitables_n_lines = 1473816

In [4]:
# with open(commoncrawl_path, 'r') as f:
#     print(len(f.readlines()))

# with open(wikitables_path, 'r') as f:
#     print(len(f.readlines()))

4986739
1473816


In [ ]:
whitespace_tokenizer = WhitespaceTokenizer()

In [53]:
with open(commoncrawl_path, 'r') as f:
    for l in f:
        _tbl_dict = ujson.loads(l)
        _example = Example.from_dict(_tbl_dict, tokenizer=None, suffix=None)
        _tbl = _example.get_table()
        print(_tbl.id)
        print(_tbl.name)
        print([_col.name for _col in _tbl.header])
        print(_tbl.data)
        print(_example.context)
        break

common-crawl/crawl-data/CC-MAIN-2015-32/segments/1438042992201.62/warc/CC-MAIN-20150728002312-00130-ip-10-236-191-2.ec2.internal.warc.gz_567892730_567906623
None
['Class', 'Interpretation']
[['0', 'Negative'], ['1', 'Equivocal'], ['2', 'Positive'], ['3', 'Positive'], ['4', 'Strongly positive'], ['5', 'Strongly positive'], ['6', 'Strongly positive']]
[['Reference Values Describes reference intervals and additional information for interpretation of test results.', 'May include intervals based on age and sex when appropriate.', 'Intervals are Mayo-derived, unless otherwise designated.', 'If an interpretive report is provided, the reference value field will state this.', 'Some individuals with clinically insignificant sensitivity to allergens may have measurable levels of IgE antibodies in serum, and results must be interpreted in the clinical context.', 'Testing for IgE antibodies is not useful in patients previously treated with immunotherapy to determine if residual clinical sensitivity

In [63]:
# CommonCrawl (1/5 of all data)

col_vocab = set()
col_set = set()
schema_item_vocab = set()
schema_item_set = set()
context_total_len = 0

with open(commoncrawl_path, 'r') as f:
    for l in tqdm(f, total=commoncrawl_n_lines):
        _tbl_dict = ujson.loads(l)
        _example = Example.from_dict(_tbl_dict, tokenizer=None, suffix=None)
        _tbl = _example.get_table()
        
        for _col in _tbl.header:
            _col_name = _col.name.lower()
            _tokens = whitespace_tokenizer.tokenize(_col_name)
            
            col_vocab.update(_tokens)
            col_set.add(_col_name)
            schema_item_vocab.update(_tokens)
            schema_item_set.add(_col_name)

        for _row in _tbl.data:
            for _item in _row:
                _item_name = _item.lower()
                _tokens = whitespace_tokenizer.tokenize(_item_name)
                
                schema_item_vocab.update(_tokens)
                schema_item_set.add(_item_name)
        
        _context = ' '.join(_example.context[0] + _example.context[1])
        context_total_len += len(_context)

In [64]:
print(f'col_vocab: Size = {len(col_vocab)}, Chars = {sum([len(w) for w in col_vocab])}')
print(f'col_set: Size = {len(col_set)}, Chars = {sum([len(w) for w in col_set])}')
print(f'schema_item_vocab: Size = {len(schema_item_vocab)}, Chars = {sum([len(w) for w in schema_item_vocab])}')
print(f'schema_item_set: Size = {len(schema_item_set)}, Chars = {sum([len(w) for w in schema_item_set])}')
print(f'context_total_len: {context_total_len}')


col_vocab: Size = 161640, Chars = 1230049
col_set: Size = 359466, Chars = 5652630
schema_item_vocab: Size = 7723528, Chars = 77095468
schema_item_set: Size = 15817619, Chars = 324494966
context_total_len: 3459606926


In [66]:
commoncrawl_n_lines * 128 / 1e6 * 16

10212.841472

In [58]:
# WikiTables

col_vocab = set()
col_set = set()
schema_item_vocab = set()
schema_item_set = set()
context_total_len = 0

with open(wikitables_path, 'r') as f:
    for l in tqdm(f, total=wikitables_n_lines):
        _tbl_dict = ujson.loads(l)
        _example = Example.from_dict(_tbl_dict, tokenizer=None, suffix=None)
        _tbl = _example.get_table()
        
        for _col in _tbl.header:
            _col_name = _col.name.lower()
            _tokens = whitespace_tokenizer.tokenize(_col_name)
            
            col_vocab.update(_tokens)
            col_set.add(_col_name)
            schema_item_vocab.update(_tokens)
            schema_item_set.add(_col_name)

        for _row in _tbl.data:
            for _item in _row:
                _item_name = _item.lower()
                _tokens = whitespace_tokenizer.tokenize(_item_name)
                
                schema_item_vocab.update(_tokens)
                schema_item_set.add(_item_name)
        
        _context = ' '.join(_example.context[0] + _example.context[1])
        context_total_len += len(_context)

In [62]:
print(f'col_vocab: Size = {len(col_vocab)}, Chars = {sum([len(w) for w in col_vocab])}')
print(f'col_set: Size = {len(col_set)}, Chars = {sum([len(w) for w in col_set])}')
print(f'schema_item_vocab: Size = {len(schema_item_vocab)}, Chars = {sum([len(w) for w in schema_item_vocab])}')
print(f'schema_item_set: Size = {len(schema_item_set)}, Chars = {sum([len(w) for w in schema_item_set])}')
print(f'context_total_len: {context_total_len}')


col_vocab: Size = 215012, Chars = 1849496
col_set: Size = 463263, Chars = 8753202
schema_item_vocab: Size = 10336239, Chars = 99786431
schema_item_set: Size = 22218910, Chars = 645300366
context_total_len: 380041729


In [67]:
wikitables_n_lines * 128 / 1e6 * 16

3018.375168

### Temp

In [46]:
WhitespaceTokenizer().tokenize("The temperature in the O'Reilly & Arbuthnot-Smythe server's main rack is 40.5 degrees.")

['The',
 'temperature',
 'in',
 'the',
 "O'Reilly",
 '&',
 'Arbuthnot-Smythe',
 "server's",
 'main',
 'rack',
 'is',
 '40.5',
 'degrees.']